# 1. Libraries

In [ ]:
# Aca se importan las librerias que vas a usar
import sys
import os
directory_path = os.path.abspath(os.path.join('..'))
utils_path = os.path.abspath(os.path.join('../utils'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    sys.path.append(utils_path)


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
from sklearn import metrics
from scipy import stats
#from utils.Validator import *
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 2. Data Collection

In [ ]:
# Aca se lee los archivos del modelamiento que deben estar en la siguiente ruta: "../data/modeling/raw.csv"
df = pd.read_csv(r"../data/modeling/03_train.csv",sep="|", dtype={'codunicocli_limpio':'str',})   
df.head()

# Escalon1. Poblacion sin riesgo

In [ ]:
#Por definición conceptual se excluye los clientes con montos de remesas acumuladas por debajo de $200
df00=df[df['MONTO_TOTAL']>=200]

# Escalon3

In [ ]:
#Idenfica alertas en el 3er escalón - Directo a evaluación
df0=df[(df['CTD_TRX']>=25) | (df['MONTO_TOTAL']>=20000)]
df0["IF_LABEL"]=3
df0.count()

In [ ]:
#df0.to_excel(r"../reports/others/Salida_Elefantes.xlsx",index=False)

# Escalon 2

In [ ]:
# Identifica a los clientes para la segmentación
df1=df00[(df00['CTD_TRX']<25) & (df00['MONTO_TOTAL']<20000)]
df1.count()

In [ ]:
df1.columns

## Algoritmo: K-means

In [ ]:
#Model 0
#Seleccion de variables para el modelo
df2 = df1[['MONTO_TOTAL', 'CTD_TRX', 'CTD_ORDENANTES', 'PORCENTAJE_MONTO']]
ds=df2.sample(50000, random_state=1)
data1_cluster = MinMaxScaler().fit_transform(ds)
k = [2,3,4,5]
for ki in k:
        outputKMeans = KMeans(n_clusters = ki, init='k-means++', random_state = 7).fit(data1_cluster)
        scoreSilhoutte = metrics.silhouette_score(data1_cluster, outputKMeans.labels_, metric='euclidean')  
        print(" k=", ki," SC=", scoreSilhoutte)

In [ ]:
#Model 1
#Seleccion de variables para el modelo
df2 = df1[['MONTO_TOTAL', 'CTD_TRX', 'CTD_ORDENANTES', 'PORCENTAJE_MONTO', 'FLG_PERFIL']]
ds=df2.sample(50000, random_state=1)
data1_cluster = MinMaxScaler().fit_transform(ds)
k = [2,3,4,5]
for ki in k:
        outputKMeans = KMeans(n_clusters = ki, init='k-means++', random_state = 7).fit(data1_cluster)
        scoreSilhoutte = metrics.silhouette_score(data1_cluster, outputKMeans.labels_, metric='euclidean')  
        print(" k=", ki," SC=", scoreSilhoutte)

In [ ]:
#Model 2
#Seleccion de variables para el modelo
df2 = df1[['MONTO_TOTAL', 'CTD_TRX', 'CTD_ORDENANTES', 'FLG_PERFIL']]
ds=df2.sample(50000, random_state=1)
data1_cluster = MinMaxScaler().fit_transform(ds)
k = [2,3,4,5]
for ki in k:
        outputKMeans = KMeans(n_clusters = ki, init='k-means++', random_state = 7).fit(data1_cluster)
        scoreSilhoutte = metrics.silhouette_score(data1_cluster, outputKMeans.labels_, metric='euclidean')  
        print(" k=", ki," SC=", scoreSilhoutte)

# Isolation Forsest

##Model0

In [ ]:
from matplotlib.pyplot import step
silhouette_list = []

df2 = df1[['MONTO_TOTAL', 'CTD_TRX', 'CTD_ORDENANTES', 'PORCENTAJE_MONTO', 'FLG_PERFIL']]


scaler = MinMaxScaler()
df2[['MONTO_TOTAL_SC', 'CTD_TRX_SC', 'CTD_ORDENANTES_SC', 'PORCENTAJE_MONTO_SC', 'FLG_PERFIL_SC']] = scaler.fit_transform(df2)
df2=df2[['MONTO_TOTAL_SC', 'CTD_TRX_SC', 'CTD_ORDENANTES_SC', 'PORCENTAJE_MONTO_SC', 'FLG_PERFIL_SC']]
df2

ds=df2.sample(50000, random_state=1)

for cont in np.arange(0.0001,0.0015,0.0003):
    cluster_labels = IsolationForest(n_estimators = 500,
                                    n_jobs=-1,
                                    max_features=(len(ds.columns)),
                                    contamination=cont,
                                    random_state=66).fit_predict(ds)
    out_n = cluster_labels[cluster_labels==-1].shape[0]
    sil = silhouette_score(ds,cluster_labels, metric='euclidean')
    silhouette_list.append(sil)
    print("silhouette for "+str(np.round(cont,4))+" with n=" +str(out_n)+" outliers is: "+str(sil))

In [ ]:
model = IsolationForest(n_estimators = 500,
                                    n_jobs=-1,
                                    max_features=(len(df2.columns)),
                                    contamination=0.0004,
                                    random_state=66).fit_predict(df2)

In [ ]:
df1['IF_LABEL']=model

In [ ]:
df1[df1["IF_LABEL"]==-1].describe()

In [ ]:
df1[df1["IF_LABEL"]==1].describe()

#Models1

In [ ]:
df2 = df1[['MONTO_TOTAL', 'CTD_TRX', 'CTD_ORDENANTES', 'FLG_PERFIL']]

scaler = MinMaxScaler()
df2[['MONTO_TOTAL_SC', 'CTD_TRX_SC', 'CTD_ORDENANTES_SC', 'FLG_PERFIL_SC']] = scaler.fit_transform(df2)
df2=df2[['MONTO_TOTAL_SC', 'CTD_TRX_SC', 'CTD_ORDENANTES_SC', 'FLG_PERFIL_SC']]
df2

In [ ]:
from matplotlib.pyplot import step
silhouette_list = []

ds=df2.sample(50000, random_state=1)

for cont in np.arange(0.0001,0.0015,0.0003):
    cluster_labels = IsolationForest(n_estimators = 500,
                                    n_jobs=-1,
                                    max_features=(len(ds.columns)),
                                    contamination=cont,
                                    random_state=66).fit_predict(ds)
    out_n = cluster_labels[cluster_labels==-1].shape[0]
    sil = silhouette_score(ds,cluster_labels, metric='euclidean')
    silhouette_list.append(sil)
    print("silhouette for "+str(np.round(cont,4))+" with n=" +str(out_n)+" outliers is: "+str(sil))

In [ ]:
# Modelo Final
model = IsolationForest(n_estimators = 500,
                                    n_jobs=-1,
                                    max_features=(len(df2.columns)),
                                    contamination=0.0001,
                                    random_state=66).fit(df2)

In [ ]:
labels=model.predict(df2)

In [ ]:
#calculo de la silueta con toda la población
sil = silhouette_score(df2,labels, metric='euclidean')


In [ ]:
sil

In [ ]:
#Guardamos el modelo
import pickle
pickle.dump(model, open(r"../src/02_models/IF.model", 'wb'))

In [ ]:
df1['IF_LABEL']=model.predict(df2)

In [ ]:
df1[df1["IF_LABEL"]==-1].describe()

In [ ]:
df1[df1["IF_LABEL"]==1].describe()

In [ ]:
#Alertas según el modelo Isolations 

df3=df1[df1["IF_LABEL"]==-1]

#df3.to_excel(r"../reports/others/Escalon22.xlsx",index=False)

In [ ]:
#Unimos las alertas del escalón 3 y 2
alertas_tot=pd.concat([df0, df3], ignore_index=True)
alertas_tot